In [1]:
import numpy as np
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import Trainer, TrainingArguments
from datasets import Dataset

In [2]:
texts = ["I am so excited about this achievement!", 
         "Humbled to announce my new job", 
         "Normal day, normal post",
         "This is just a regular update.",
         "Thrilled to share I got a promotion!"]

labels = [1, 1, 0, 0, 1]  # 1=brag, 0=not brag

data = Dataset.from_dict({"text": texts, "label": labels})

model_name = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [3]:
def tokenize(batch):
    return tokenizer(batch["text"], truncation=True, padding="max_length", max_length=128)

data = data.map(tokenize, batched=True)
data.set_format("torch", columns=["input_ids", "attention_mask", "label"])

Map:   0%|          | 0/5 [00:00<?, ? examples/s]

In [4]:
split = data.train_test_split(test_size=0.2)
train_ds = split["train"]
test_ds = split["test"]

# 3. Load model
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [5]:
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=1,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    logging_steps=10,
    evaluation_strategy="epoch"
)

# 5. Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=test_ds
)

# Train
trainer.train()

C:\Users\Sagar\AppData\Local\Programs\Python\Python310\lib\site-packages\transformers\training_args.py:1545: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,No log,0.678902


TrainOutput(global_step=2, training_loss=0.7067469358444214, metrics={'train_runtime': 6.1839, 'train_samples_per_second': 0.647, 'train_steps_per_second': 0.323, 'total_flos': 132467398656.0, 'train_loss': 0.7067469358444214, 'epoch': 1.0})

In [6]:
dummy_input = tokenizer("Hello world!", return_tensors="pt")
torch.onnx.export(
    model,
    (dummy_input["input_ids"], dummy_input["attention_mask"]),
    "model.onnx",
    input_names=["input", "mask"],
    output_names=["output"],
    dynamic_axes={"input": {0: "batch_size"}, "mask": {0: "batch_size"}, "output": {0: "batch_size"}},
    opset_version=11
)

print("Exported model to ONNX: model.onnx")

C:\Users\Sagar\AppData\Local\Programs\Python\Python310\lib\site-packages\transformers\models\distilbert\modeling_distilbert.py:215: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  mask, torch.tensor(torch.finfo(scores.dtype).min)


Exported model to ONNX: model.onnx
